In [4]:
%pip install python-dotenv


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from dotenv import load_dotenv
import os
load_dotenv()

kakao_api_key = os.getenv("KAKAOAPI")
google_api_key = os.getenv("GOOGLEAPI")


In [22]:
import pandas as pd

# 파일 경로
file_path = "D:/바탕화면/대학수업/3-2/빅데프/실습/Term_Project/CarrotMarket/originaldata/서울교통공사_역주소 및 전화번호_20230331.csv"

# CSV 파일 읽기 (인코딩 명시)
data = pd.read_csv(file_path, encoding='euc-kr')

# "img" 컬럼 추가 및 값 설정
data["img"] = [f"subway{i}" for i in range(1, len(data)+1)]

# 데이터 확인
print(data.head())


   연번  역번호  호선    역명         역전화번호                       도로명주소  \
0   1  150   1    서울  02-6110-1331  서울특별시 중구 세종대로 지하2(남대문로 5가)   
1   2  151   1    시청  02-6110-1321     서울특별시 중구 세종대로 지하101(정동)   
2   3  152   1    종각  02-6110-1311     서울특별시 종로구 종로 지하55(종로1가)   
3   4  153   1  종로3가  02-6110-1301    서울특별시 종로구 종로 지하129(종로3가)   
4   5  154   1  종로5가  02-6110-1291    서울특별시 종로구 종로 지하216(종로5가)   

                             지번주소      img  
0   서울특별시 중구 남대문로5가 73-6 서울역(1호선)  subway1  
1        서울특별시 중구 정동 5-5 시청역(1호선)  subway2  
2      서울특별시 종로구 종로1가 54 종각역(1호선)  subway3  
3  서울특별시 종로구 종로3가 10-5 종로3가역(1호선)  subway4  
4  서울특별시 종로구 종로5가 82-1 종로5가역(1호선)  subway5  


In [23]:
import pandas as pd
import requests
from urllib.parse import urlparse
import os
# 도로명 주소 column 이름
address_column = '도로명주소'

# 결과를 저장할 리스트
result_list = []
kakao_api_key = os.getenv("KAKAOAPI")
# 모든 도로명 주소에 대해 반복
for address in data[address_column]:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?&query=' + address
    result = requests.get(urlparse(url).geturl(), headers={'Authorization': f'KakaoAK {kakao_api_key}'}).json()
    
    # 주소에 해당하는 데이터 찾기
    matched_data = data[data[address_column] == address]

    match_first=result['documents'][0]['address']
    lat = float (match_first['y'])
    lng = float (match_first['x'])
    result_list.append((address, lat, lng))
        
# 결과 출력
for result in result_list:
    address, lat, lng = result
    print(f"도로명 주소: {address}, 위도: {lat}, 경도: {lng}")

도로명 주소: 서울특별시 중구 세종대로 지하2(남대문로 5가), 위도: 37.5571594835988, 경도: 126.972554606456
도로명 주소: 서울특별시 중구 세종대로 지하101(정동), 위도: 37.5654385176408, 경도: 126.976983237511
도로명 주소: 서울특별시 종로구 종로 지하55(종로1가), 위도: 37.5702140381269, 경도: 126.983239678933
도로명 주소: 서울특별시 종로구 종로 지하129(종로3가), 위도: 37.5704266025836, 경도: 126.99202930223
도로명 주소: 서울특별시 종로구 종로 지하216(종로5가), 위도: 37.5709075371016, 경도: 127.001916647342
도로명 주소: 서울특별시 종로구 종로 지하302(창신동), 위도: 37.5717775660734, 경도: 127.011167102297
도로명 주소: 서울특별시 동대문구 왕산로 지하1(신설동), 위도: 37.5747187974165, 경도: 127.025093714935
도로명 주소: 서울특별시 동대문구 왕산로 지하93(제기동), 위도: 37.5781967284885, 경도: 127.034690599752
도로명 주소: 서울특별시 동대문구 왕산로 지하205(전농동), 위도: 37.5802331563072, 경도: 127.045085886324
도로명 주소: 서울특별시 종로구 종로 359(숭인동), 위도: 37.5733722075697, 경도: 127.01675118037
도로명 주소: 서울특별시 중구 서소문로 지하127(서소문동), 위도: 37.5635793508101, 경도: 126.975389363157
도로명 주소: 서울특별시 중구 을지로 지하42(을지로1가), 위도: 37.5660509513873, 경도: 126.982372901787
도로명 주소: 서울특별시 중구 을지로 지하106(을지로3가), 위도: 37.56626594615, 경도: 126.990308356253
도로명 주

In [24]:
import pandas as pd

# 결과를 저장할 DataFrame 생성
result_df = pd.DataFrame(result_list, columns=[address_column, 'lat', 'lng'])

# 원래 데이터와 병합
merged_df = pd.merge(data, result_df, on=address_column)

# '분류' 열 추가하고, 값은 모두 'subway'로 설정
merged_df['category'] = 'subway'

# 새로운 CSV 파일로 저장
merged_df.to_csv("D:/바탕화면/대학수업/3-2/빅데프/실습/Term_Project/CarrotMarket/outputData/Subwaylocation.csv", index=False, encoding='euc-kr')


In [25]:
import pandas as pd

# 파일 경로
file_path = "D:/바탕화면/대학수업/3-2/빅데프/실습/Term_Project/CarrotMarket/outputData/Subwaylocation.csv"

# CSV 파일 읽기 (인코딩 명시)
data = pd.read_csv(file_path, encoding='euc-kr')

# 데이터 확인
print(data.head())


   연번  역번호  호선    역명         역전화번호                       도로명주소  \
0   1  150   1    서울  02-6110-1331  서울특별시 중구 세종대로 지하2(남대문로 5가)   
1   2  151   1    시청  02-6110-1321     서울특별시 중구 세종대로 지하101(정동)   
2   3  152   1    종각  02-6110-1311     서울특별시 종로구 종로 지하55(종로1가)   
3   4  153   1  종로3가  02-6110-1301    서울특별시 종로구 종로 지하129(종로3가)   
4   5  154   1  종로5가  02-6110-1291    서울특별시 종로구 종로 지하216(종로5가)   

                             지번주소      img        lat         lng category  
0   서울특별시 중구 남대문로5가 73-6 서울역(1호선)  subway1  37.557159  126.972555   subway  
1        서울특별시 중구 정동 5-5 시청역(1호선)  subway2  37.565439  126.976983   subway  
2      서울특별시 종로구 종로1가 54 종각역(1호선)  subway3  37.570214  126.983240   subway  
3  서울특별시 종로구 종로3가 10-5 종로3가역(1호선)  subway4  37.570427  126.992029   subway  
4  서울특별시 종로구 종로5가 82-1 종로5가역(1호선)  subway5  37.570908  127.001917   subway  


In [27]:
import csv
import os
import requests

google_api_key = os.getenv("GOOGLEAPI")

# CSV 파일 경로 설정
file_path = 'D:/바탕화면/대학수업/3-2/빅데프/실습/Term_Project/CarrotMarket/outputData/Subwaylocation.csv'

# 이미지 저장 디렉토리 경로 설정
image_directory = 'D:/바탕화면/대학수업/3-2/빅데프/실습/Term_Project/CarrotMarket/outputData/SubwayImg'

# CSV 파일 열기
with open(file_path, 'r', encoding='euc-kr') as file:
    # CSV 파일을 읽는 reader 생성
    reader = csv.reader(file)

    # 첫 번째 줄을 헤더로 읽음
    headers = next(reader)

    # 'lat' 컬럼과 'lng' 컬럼의 인덱스를 찾기
    lat_index = headers.index('lat')
    lng_index = headers.index('lng')
    count_index = headers.index('img')

    # 각 줄에 대한 작업 수행
    for row in reader:
        # 해당 줄의 'lat' 컬럼과 'lng' 컬럼 값 출력
        print(row[count_index], row[lat_index], row[lng_index])

        url = f"https://maps.googleapis.com/maps/api/streetview?size=600x300&location={row[lat_index]},{row[lng_index]}&fov=80&heading=70&pitch=0&key={google_api_key}"
        response = requests.get(url)

        if response.status_code == 200:
            # 이미지 파일 이름 지정
            file_name = f"{row[count_index]}.jpg"
            # 이미지 파일의 저장 경로
            file_path = os.path.join(image_directory, file_name)
            with open(file_path, 'wb') as img_file:
                img_file.write(response.content)
        else:
            print("Error occurred while downloading image.")


subway1 37.5571594835988 126.972554606456
subway2 37.5654385176408 126.976983237511
subway3 37.5702140381269 126.983239678933
subway4 37.5704266025836 126.99202930223
subway5 37.5709075371016 127.001916647342
subway6 37.5717775660734 127.011167102297
subway7 37.5747187974165 127.025093714935
subway8 37.5781967284885 127.034690599752
subway9 37.5802331563072 127.045085886324
subway10 37.5733722075697 127.01675118037
subway11 37.5635793508101 126.975389363157
subway12 37.5660509513873 126.982372901787
subway13 37.56626594615 126.990308356253
subway14 37.5666369941572 126.997852780281
subway15 37.5655895229776 127.009131498424
subway16 37.5656540317328 127.019540574485
subway17 37.5644687111181 127.029031221392
subway18 37.5612217415191 127.036655378283
subway18 37.5612217415191 127.036655378283
subway151 37.5612217415191 127.036655378283
subway151 37.5612217415191 127.036655378283
subway19 37.55557788354 127.043584493543
subway20 37.5471882244091 127.047365178682
subway21 37.544502398398